## ダウンロードしたデータの結合

前項でダウンロードしたデータは、日付ごとに分かれているので、1つのファイルにまとめます。

In [ ]:
import re
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from os import path
from pathlib import Path

In [ ]:
DATA_DIR = 'C:/sample/data/download_data'  # データをダウンロードしたフォルダ
PRICE_DIR = path.join(DATA_DIR, 'japan-all-stock-prices')  # 株価一覧表
STOCK_DATA_DIR = path.join(DATA_DIR, 'japan-all-stock-data')  # 投資指標データ
FIN_RESULTS_DIR = path.join(DATA_DIR, 'japan-all-stock-financial-results')  # 決算・財務・業績データ
OUT_DIR = 'C:/sample/data'  # 結合したデータを保存するフォルダ

In [ ]:
price_data = []
file_list = list(Path(PRICE_DIR).glob('*.csv'))  # PRICE_DIR以下にあるcsvファイルのパスをすべて取得する

for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]  # カラム名にスペースが混じることがあるので削除
    timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])  # 日時はファイル名から取得
    price_data.append(data_.assign(日時=timestamp))

price_data = pd.concat(price_data).assign(
    日時=lambda x: x['日時'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    )
)

In [ ]:
stock_data = []
file_list = list(Path(STOCK_DATA_DIR).glob('*.csv'))  # STOCK_DATA_DIR以下にあるcsvファイルのパスをすべて取得する

for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]  # カラム名にスペースが混じることがあるので削除
    timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])  # 日時はファイル名から取得
    stock_data.append(data_.assign(日時=timestamp))

stock_data = pd.concat(stock_data).assign(
    日時=lambda x: x['日時'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    ),
    安値日付=lambda x: x['安値日付'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    ),
    高値日付=lambda x: x['高値日付'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 時刻をすべて00:00:00に合わせる
    )
)

In [ ]:
daily_data = pd.merge(price_data, stock_data, on=['SC', '時価総額（百万円）', '名称', '市場', '業種', '日時'], how='left')

In [ ]:
financial_data = []
file_list = list(Path(FIN_RESULTS_DIR).glob('*.csv'))
for file in tqdm(file_list):
    data_ = pd.read_csv(f'file:{file}', encoding='sjis', na_values='-')
    data_.columns = [column.strip() for column in data_.columns]
    timestamp = pd.Timestamp(re.findall(r'\d{8}', file.stem)[0])
    financial_data.append(data_.assign(日時=timestamp))

financial_data = pd.concat(financial_data)
financial_data = financial_data.assign(
    日時=lambda x: x['日時'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())
    ),
    kessan_tmp=lambda x: x['決算発表日（本決算）'].map(
        lambda elm: pd.Timestamp(pd.to_datetime(elm).date())  # 全角()が扱えないので一時カラムで対処
    )
).drop(columns='決算発表日（本決算）').rename(columns={'kessan_tmp': '決算発表日（本決算）'})[financial_data.columns]  # 列の順番を元に戻す

結合したデータを保存します。pickleという形式で保存することでデータの型などのメタ情報をそのまま保存できます。

In [ ]:
daily_data.to_pickle(path.join(OUT_DIR, 'daily_data.pickle'))
financial_data.to_pickle(path.join(OUT_DIR, 'financial_data_all.pickle'))